# how much playing time does it take to get from lvl 1 to lvl 60

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
import pandas as pd
from woc import data, outliers

In [4]:
clean_df = (
    data.load('../data/wowah_data.csv')
    .pipe(outliers.remove, min_lvl=20)
)

DEBUG:woc.data:[_load_csv] shape=(100000, 7) time=0:00:00.206799
DEBUG:woc.data:[_normalize_colnames] shape=(100000, 7) time=0:00:00.005105
DEBUG:woc.data:[_parse_dtypes] shape=(100000, 7) time=0:00:02.619103
INFO:woc.data:[load] shape=(100000, 7) time=0:00:02.835622
DEBUG:woc.outliers:[_remove_below_min_level] shape=(93864, 7) time=0:00:00.009130
INFO:woc.outliers:[remove] shape=(93864, 7) time=0:00:00.009956


In [6]:
clean_df.head()

,char,level,race,charclass,zone,guild,timestamp
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09
5,2289,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09
6,61239,68,Orc,Hunter,Blade's Edge Mountains,243,2008-01-01 00:02:14
7,59772,69,Orc,Warrior,Shadowmoon Valley,35,2008-01-01 00:02:14


In [75]:
%%timeit
clean_df.groupby('char')['level'].apply(lambda s: s.max())

447 ms ± 28.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
%%timeit
clean_df.groupby('char')[['level']].max()

7 ms ± 304 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [84]:
%%timeit

(
    clean_df
    .groupby('char')
    .agg({
        'level': np.max
    })
)

7.13 ms ± 238 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
import numpy as np
chars_above_min_lvl = (
    clean_df
    .groupby('char')
    .agg(
        max_lvl=('level', np.max)
    )
    .loc[lambda d: d['max_lvl'] > 10]
)

clean_df.loc[lambda d: d['char'].isin(chars_above_min_lvl.index)]

,char,level,race,charclass,zone,guild,timestamp
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09
5,2289,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09
6,61239,68,Orc,Hunter,Blade's Edge Mountains,243,2008-01-01 00:02:14
7,59772,69,Orc,Warrior,Shadowmoon Valley,35,2008-01-01 00:02:14
...,...,...,...,...,...,...,...
99995,58284,66,Blood Elf,Mage,Terokkar Forest,165,2008-01-03 16:25:34
99996,62379,67,Blood Elf,Rogue,Nagrand,174,2008-01-03 16:25:34
99997,64813,67,Blood Elf,Priest,Netherstorm,-1,2008-01-03 16:25:34
99998,59601,68,Blood Elf,Mage,Shadowmoon Valley,216,2008-01-03 16:25:39


In [101]:
chars_above_min_lvl

,max_lvl
char,
9,70
21,70
23,70
28,70
36,70
...,...
65623,11
65631,30
65672,70


In [ ]:
df[df['col'] > ...]

In [93]:
max_lvl_per_char

,max_lvl
char,
9,70
21,70
23,70
28,70
36,70
...,...
65690,1
65691,1
65692,1


In [ ]:
clean_df = parse_dtypes(
    normalize_colnames(df)
)

take vegetables
 - **season** with *salt*
 - **roast** them in the *oven*
 - **serve** them on a *plate*
 
we don't say

serve(
    roast(
        season(
            vegetables
        )
     )
)

In [67]:
df.dtypes

char           int64
 level         int64
 race         object
 charclass    object
 zone         object
 guild         int64
 timestamp    object
dtype: object

In [68]:
clean_df.dtypes

char                  int64
level                 int64
race               category
charclass          category
zone                 object
guild                 int64
timestamp    datetime64[ns]
dtype: object